# Submission API Testing Starter Pack

CRDS env vars need to be set on the command line before starting the notebook:

```bash
$ export CRDS_PATH=/home/developer/cache_volumes/crds-cache-default-test
$ export CRDS_SERVER_URL=https://roman-crds-dev.stsci.edu
$ export CRDS_CONTEXT=roman_0039.pmap
```

For submission testing, you may also want to create and set a MAST auth token:

https://auth.mast.stsci.edu/token

```bash
export MAST_API_TOKEN="1234567abcde"
```

In [ ]:
# IMPORT LIBRARIES

import os
import crds
from crds import log
from crds.submit import rc_submit
from pprint import pprint as pp

In [ ]:

log.set_verbose(0)   # 80 for debugging web activity, 0 for no debug, 50 for basic flow

In [ ]:
# ENVIRONMENT SETUP

HOME = %pwd
CACHE_VOL = os.path.join(HOME, "cache_volumes")
PROJECT = 'roman'   # hst, jwst, roman
SERVER = 'dev'    # dev, test, cit
CONTEXT = f"{PROJECT}-edit" # -edit, -operational, or _number like "_0038"

#https://auth.mast.stsci.edu/token
MAST_API_TOKEN = os.environ["MAST_API_TOKEN"]

# use the test suite cache or create a new folder
CRDS_PATH = os.path.join(HOME, "cache_volumes", "crds-cache-default-test")
APICACHE = os.path.join(CACHE_VOL, "apicache")
TESTDATA = os.path.join(CACHE_VOL, "testdata")
os.makedirs(APICACHE, exist_ok=True)
os.makedirs(TESTDATA, exist_ok=True)


os.environ["CRDS_SERVER_URL"] = "https://" + PROJECT + "-crds-" + SERVER + ".stsci.edu"
os.environ["CRDS_PATH"] = CRDS_PATH # or APICACHE

In [ ]:
# CRDS STATUS CHECK

! crds list --status

# CRDS Summary = 'The __rationale__ variable is no longer maintained and will be removed in a future release.'
# CRDS Version = '11.16.15.dev3+g739ffdd3.d20221018, b11.4.0, 64d96076d89b32a5687a6b77bb910ab93b3a99b3'
# CRDS_MODE = 'auto'
# CRDS_PATH = '/home/developer/cache_volumes/crds-cache-default-test'
# CRDS_SERVER_URL = 'https://roman-crds-dev.stsci.edu'
# Cache Locking = 'enabled, multiprocessing'
# Effective Context = 'roman_0039.pmap'
# Last Synced = '2022-10-18 18:49:15.168687'
# Python Executable = '/home/developer/venv/bin/python3'
# Python Version = '3.9.2.final.0'
# Readonly Cache = False

In [ ]:
# TRY SYNCING CACHE WITH SERVER
# CONTEXT = roman-edit
! crds sync --contexts CONTEXT --repair-files --purge-mappings --purge-references

# CRDS - INFO -  Symbolic context 'roman-edit' resolves to 'roman_0044.pmap'
# CRDS - INFO -  Removing all context pickles.  Use --save-pickles to recreate for specified contexts.
# CRDS - INFO -  Fetching  /home/developer/cache_volumes/crds-cache-default-test/mappings/roman/roman_wfi_distortion_0008.rmap    3.8 K bytes  (1 / 3 files) (0 / 4.9 K bytes)
# CRDS - INFO -  Fetching  /home/developer/cache_volumes/crds-cache-default-test/mappings/roman/roman_wfi_0042.imap      741 bytes  (2 / 3 files) (3.8 K / 4.9 K bytes)
# CRDS - INFO -  Fetching  /home/developer/cache_volumes/crds-cache-default-test/mappings/roman/roman_0044.pmap      364 bytes  (3 / 3 files) (4.6 K / 4.9 K bytes)
# CRDS - INFO -  0 errors
# CRDS - INFO -  0 warnings
# CRDS - INFO -  5 infos

## Submission Testing

1. Create submission object
2. Update fields
3. Add file(s)
4. Submit
5. (optional) Confirm submission and check filemap

In [ ]:
# 1. create a submission object

submission = rc_submit.Submission(PROJECT, SERVER)

In [ ]:
# 2. update fields

submission["deliverer"] = "rkein"
submission["other_email"] = ""
submission["file_type"] = "distortion"
submission["history_updated"] = True
submission["descrip_updated"] = True
submission["useafter_updated"] = True
submission["useafter_matches"] = True
submission["pedigree_updated"] = True
submission["keywords_checked"] = True
submission["compliance_verified"] = True
submission["ingest_files"] = True
submission["etc_delivery"] = False
submission["calpipe_version"] = "xxx, 1.0"
submission["replacement_files"] = False
submission["old_reference_files"] = "xxx"
submission["replacing_badfiles"] = False
submission["jira_issue"] = ""
submission["table_rows_changed"] = "xxx"
submission["modes_affected"] = "xxxx"
submission["correctness_testing"] = "lots"
submission["additional_considerations"] = "a few"
submission["change_level"] = "SEVERE"
submission["description"] = "roman submission api test"
submission["instrument"] = "wfi"

In [ ]:
# 3. add file(s)
f = f"{APICACHE}/roman/roman_wfi_distortion_nine.asdf"
submission.add_file(f)

In [ ]:
result = submission.submit()

In [ ]:
# submission.update_filemap()
submission.file_map

# {'roman_wfi_distortion_nine.asdf': 'roman_wfi_distortion_0028.asdf',
#  'roman_0048.pmap': 'roman_0049.pmap',
#  'roman_wfi_0046.imap': 'roman_wfi_0047.imap',
#  'roman_wfi_distortion_0012.rmap': 'roman_wfi_distortion_0013.rmap'}